# Analyse en composantes principales sur données quotidiennes de Z500

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - février 2022**

L'analyse en composantes principales (ACP) ou Empirical Orthogonal Function (EOF) analysis en anglais est une méthode statistique permettant de simplifier des données spatio-temporelles en les décomposant structures spatiales de variabilité et en projections temporelles de ces structures. Les vecteurs propres (Empirical Orthogonal Functions = EOFs) sont des fonctions de base en terme de variance et correspodent à des structures spatiales. Les projections temporelles associées à ces structures spatiales sont les composantes principales (Principal Components = PCs) et correspondent aux coefficients temporels des EOFs.

On utilisera ici le package Python eofs : https://ajdawson.github.io/eofs/latest/

Note : on n'utilise pas ici la methode solver.reconstruction car bug avec la version 3.1 de Python : https://github.com/ajdawson/eofs/commit/416ea0c135485ddd2cdf8e6d2b9f51a4d1cddc29

Concepts Python illustrés :

- Exploitation de fichiers netcdf (xarray)
- Tracé de cartes (cartopy)
- Manipulation de séries temporelles (pandas)
- Analyse en composantes principales (eofs)

In [ ]:
%matplotlib inline

import os

import numpy as np

import xarray as xr

from cartopy import config
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.path as mpath

import pandas as pd
from pandas import Series

from eofs.standard import Eof

In [ ]:
dir_data='./data/'
dir_figs='./figs/'
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)

# Traitement des données quotidiennes

In [ ]:
year1='1980-01-01'
year2='2020-12-31'
latS=20
latN=80

nc_file = dir_data+'hgt500.1980-2021.nc'
data    = xr.open_dataset(nc_file).sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))
#data=data.groupby('time.dayofyear') - data.groupby('time.dayofyear').mean('time')
print(data)

dates = pd.date_range(year1, year2, freq='D')
print(dates)

In [ ]:
selection =['Year', 'January', 'February', 'March', 'April', 'May','June',
     'July', 'August', 'September', 'October', 'November', 'December']
select=int(input("Entrer le nom du mois, entrer 0 pour l'année complète : "))
sel_text=selection[select]

if select == 0:
    months=np.any([dates.month==1,
                   dates.month==2,
                   dates.month==2,
                   dates.month==4,
                   dates.month==5,
                   dates.month==6,
                   dates.month==7,
                   dates.month==8,
                   dates.month==9,
                   dates.month==10,
                   dates.month==11,
                   dates.month==12],axis=0)
else:
    months=np.any([dates.month==select],axis=0)
    
dates2=dates[months]
print(dates2)

data_month = data.sel(lat=slice(latN,latS)).sel(time=dates2)
print(data_month)
z0 = data_month['hgt']
lat  = data_month.lat.values
time  = data_month.time.values

In [ ]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[ lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
z=lonflip(z0)
print(z.shape)
lon  = z.lon.values

lonW=-80
lonE=30
lon1 = list(lon).index(lonW)
lon2 = list(lon).index(lonE)

z=z[:,:,lon1:lon2+1]
print(z.shape)
lon=lon[lon1:lon2+1]

zmean=z.mean(axis=0)

# Tracé du champ moyen

In [ ]:
projection = ccrs.Orthographic(central_longitude=(lonW+lonE)/2, central_latitude=(lat[-1]+lat[0])/2)

def make_boundary_path(lon,lat):
    lons,lats=np.meshgrid(lon,lat)
    boundary_path = np.array([lons[-1,:],lats[-1,:]])
    boundary_path = np.append(boundary_path,np.array([lons[::-1,-1],lats[::-1,-1]]),axis=1)
    boundary_path = np.append(boundary_path,np.array([lons[1,::-1],lats[1,::-1]]),axis=1)
    boundary_path = np.append(boundary_path,np.array([lons[:,1],lats[:,1]]),axis=1)
    boundary_path = mpath.Path(np.swapaxes(boundary_path, 0, 1))
    return boundary_path

def plot_map(ax):
    ax.coastlines()
    ax.gridlines()    
    boundary_path = make_boundary_path(lon, lat)
    ax.set_boundary(boundary_path, transform=ccrs.PlateCarree())
    return ax

In [ ]:
levels = np.arange(4800,6200,50)
cmap = 'jet'

fig = plt.figure(figsize=(12,7))
fig.suptitle('Geopotential height (mgp) at 500 hPa', fontsize=16)

ax = fig.add_subplot(111, projection=projection)
ax.set_title(sel_text, fontsize=14)
plot_map(ax)
p1 = ax.contourf(lon, lat, zmean, levels, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, zmean, levels, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(p1, orientation='vertical', shrink=0.74, pad=0.1)

plt.show()

figname=dir_figs+'z500_mean_'+sel_text
fig.savefig(figname+'.png', bbox_inches='tight')

# Analyse en composantes principales

In [ ]:
wgts = np.sqrt(np.cos(np.deg2rad(lat)))[:, np.newaxis]

In [ ]:
z=np.array(z)
solver = Eof(z, weights=wgts, center=True)
#solver = Eof(z, weights=None, center=True) # ACP sans pondération

In [ ]:
eigenvalues = solver.eigenvalues()
total_variance = solver.totalAnomalyVariance()
varfrac = solver.varianceFraction()
eofs = solver.eofs()
pcs = solver.pcs(pcscaling=0)

print(eofs.shape)
print(pcs.shape)
print(np.min(eofs))
print(np.max(eofs))

print('******** Nombre de valeurs propres **********')
print(eigenvalues.size)
print('******** valeurs propres **********')
print(eigenvalues)
print('******** variance totale (somme des valeurs propres) **********')
print(total_variance)
print('******** Variance PC1 **********')
print(np.var(pcs[:,0]))
print('******** Variance PC2 **********')
print(np.var(pcs[:,1]))
print('******** Variance PC3 **********')
print(np.var(pcs[:,2]))
print('******** correlation PC1 PC2 **********')
cor_pc12=np.corrcoef(pcs[:,0], pcs[:,1])
print(round(cor_pc12[0,1],2))
print('******** pourcentage de variance EOF1 **********')
print(varfrac[0]*100)
print('******** pourcentage de variance EOF2 **********')
print(varfrac[1]*100)

In [ ]:
axes_class = (GeoAxes, dict(map_projection=projection))

fig = plt.figure(figsize=(15,6))
fig.suptitle('Geopotential height at 500 hPa - EOFs : '+sel_text, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(3, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

levs = np.arange(-0.1, 0.12, 0.02)
                   
for i, ax in enumerate(axgr):
    plot_map(ax)
    var=round(varfrac[i]*100,2)
    ax.set_title('EOF'+str(i+1)+' ('+str(var)+'%)', fontsize=10, loc='center')
    cf = ax.contourf(lon, lat, eofs[i], levs, transform=ccrs.PlateCarree(), cmap='bwr', extend='both')
    c = ax.contour(lon, lat, eofs[i], levs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

plt.show()

figname=dir_figs+'eofs_z500_'+sel_text
fig.savefig(figname+'.png', bbox_inches='tight')

fig = plt.figure(figsize=(15,6))
fig.suptitle('Geopotential height at 500 hPa - PCs : '+sel_text, fontsize=16)

plt.subplots_adjust(hspace=0.5, wspace=0.4)

for i in range(0, 15):
        plt.subplot(3, 5, i+1)
        plt.title('PC'+str(i+1)+'(t)')
        plt.axhline(0, color='k', linewidth=0.5)
        plt.ylim(-5000, 5000)
        plt.plot(pcs[:,i], color='k', linewidth=0.5, alpha=0.7)

plt.show()

figname=dir_figs+'pcs_z500_'+sel_text
fig.savefig(figname+'.png', bbox_inches='tight')

fig = plt.figure(figsize=(15, 7))
fig.suptitle('Geopotential height at 500 hPa - Variance fraction : '+sel_text, fontsize=16)

eof_num = range(1, 16)
plt.bar(eof_num, varfrac[0:15], width=0.5)
plt.axhline(0, color='k')
plt.xticks(range(1, 16))
plt.xlabel('EOF #')
plt.ylabel('Variance Fraction')
plt.xlim(1, 15)
plt.ylim(np.min(varfrac), np.max(varfrac)+0.01)
plt.show()
figname=dir_figs+'varfrac_z500_'+sel_text
fig.savefig(figname+'.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.suptitle('Geopotential height at 500 hPa - EOF1 and PC1 : '+sel_text, fontsize=16)

ax = fig.add_subplot(211, projection=projection)
plot_map(ax)
var=round(varfrac[0]*100,2)
ax.set_title('EOF1'+' ('+str(var)+'%)', fontsize=10, loc='center')
cf = ax.contourf(lon, lat, eofs[0], levs, transform=ccrs.PlateCarree(), cmap='bwr', extend='both')
c = ax.contour(lon, lat, eofs[0], levs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='vertical', shrink=0.74, pad=0.1)

ax = fig.add_subplot(212)
plt.title('PC1(t)', fontsize=14)
plt.axhline(0, color='k', linewidth=0.5)
plt.ylim(-5000, 5000)
plt.plot(pcs[:,0], color='k', linewidth=0.5)

plt.show()

figname=dir_figs+'eof1_pc1_z500'+sel_text
fig.savefig(figname+'.png', bbox_inches='tight')

#  Reconstruction d'une journée particulière

In [ ]:
#--  Manage dates
time_str=[x for x in range(len(time))]
date_str=[x for x in range(len(time))]
for i in range(len(time)):
	time_str[i] = str(time[i])
	date_str[i] = time_str[i][0:10]

date=input("Entrer une date présente dans les données (format YYYY-MM-DD) : ")
idate = list(date_str).index(date)

In [ ]:
def reconst_eof_pc(eofs,pcs,n):
    reconst = 0
    for i in range(n):
        reconst+=eofs[i]*pcs[idate,i]/wgts
    return(reconst)

In [ ]:
levels = np.arange(4800,6200,50)
cmap = 'jet'

fig = plt.figure(figsize=(17,12))
fig.suptitle('Reconstruction of the geopotential height at 500 hPa (mgp) : '+date, fontsize=16)

ax = fig.add_subplot(4,3,2, projection=projection)
plot_map(ax)
ax.set_title('Full field', fontsize=14)
p1 = ax.contourf(lon, lat, z[idate,:,:], levels, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, z[idate,:,:], levels, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(p1, orientation='vertical', shrink=0.74, pad=0.1)

ax = fig.add_subplot(4,3,4, projection=projection)
plot_map(ax)
ax.set_title('1 EOF', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,1)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,1)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

ax = fig.add_subplot(4,3,5, projection=projection)
plot_map(ax)
ax.set_title('2 EOFs', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,2)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,2)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

ax = fig.add_subplot(4,3,6, projection=projection)
plot_map(ax)
ax.set_title('3 EOFs', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,3)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,3)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

ax = fig.add_subplot(4,3,7, projection=projection)
plot_map(ax)
ax.set_title('4 EOFs', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,4)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,4)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

ax = fig.add_subplot(4,3,8, projection=projection)
plot_map(ax)
ax.set_title('5 EOFs', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,5)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,5)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

ax = fig.add_subplot(4,3,9, projection=projection)
plot_map(ax)
ax.set_title('6 EOFs', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,6)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,6)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

ax = fig.add_subplot(4,3,10, projection=projection)
plot_map(ax)
ax.set_title('7 EOFs', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,7)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,7)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

ax = fig.add_subplot(4,3,11, projection=projection)
plot_map(ax)
ax.set_title('8 EOFs', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,8)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,8)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

ax = fig.add_subplot(4,3,12, projection=projection)
plot_map(ax)
ax.set_title('9 EOFs', fontsize=14)
p1 = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,9)+zmean, levels, transform=ccrs.PlateCarree(),
                 cmap=cmap, extend='both')
p2 = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,9)+zmean, levels, colors='black', linewidths=0.2,
                transform=ccrs.PlateCarree())   

plt.show()

figname=dir_figs+'reconstruction_z500_'+str(date)
fig.savefig(figname+'.png', bbox_inches='tight')

# Pour aller plus loin

Etude des régimes de temps de l'Atlantique nord (cf. calepin régimes de temps IENM1) :

- Première étape : Analyse en composantes principales des données quotidiennes quotidennes de Pmer ou de Z500 sur les mois d'hiver (DJFM) ou d'été (JJAS).
- Deuxième étape : classification automatique ("clustering") K-means dans l'espace des PCs.